In [11]:
# Standard libraries
import re
import string

# Data processing
import numpy as np
import pandas as pd

# NLP tools
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# MLflow
import mlflow
import mlflow.sklearn

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')  # Required for WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\suref\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\suref\AppData\Roaming\nltk_data...


True

In [12]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
930,I cannot get past the message of this movie. I...,negative
2,The `plot' of this film contains a few holes y...,negative
391,"Dear God! I kept waiting for this movie to ""ge...",negative
304,When I go to see movies I would stay up and wa...,negative
608,If you are in to bad movies for the entertainm...,negative


In [13]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [14]:
df = normalize_text(df)
df.head()

,review,sentiment
930,cannot get past message movie laid much clearl...,negative
2,plot film contains hole could drive massive tr...,negative
391,dear god kept waiting movie get started waited...,negative
304,go see movie would stay watch like it would go...,negative
608,bad movie entertainment witnessing bad movie b...,negative


In [15]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [16]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [17]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
930,cannot get past message movie laid much clearl...,0
2,plot film contains hole could drive massive tr...,0
391,dear god kept waiting movie get started waited...,0
304,go see movie would stay watch like it would go...,0
608,bad movie entertainment witnessing bad movie b...,0


In [18]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [19]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [21]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/sc/capstone.mlflow')
dagshub.init(repo_owner='sc', repo_name='capstone_repo', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\SohailChougle\anaconda3\envs\atlas\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2d6cc66b-27a8-4e29-9601-fd0be9d05df0&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=735f6555bd5eb07975172e5c31c31e3e021baa8770df0567f082be450c66d8d9




Accessing as sc

Repository capstone_repo doesn't exist, creating it under current user.

Initialized MLflow to track repo "sc/capstone_repo"

Repository sc/capstone_repo initialized!

2025/09/01 12:52:02 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/ac314f60bcc24e779d9ae6514af6349f', creation_time=1756716722495, experiment_id='0', last_update_time=1756716722495, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Save model locally first, then log as an artifact (compatible with DagsHub)
        logging.info("Saving model locally...")
        model_path = "model.pkl"
        with open(model_path, "wb") as f:
            pickle.dump(model, f)
        
        logging.info("Logging model as artifact...")
        mlflow.log_artifact(model_path)
        
        # Cleanup local file
        os.remove(model_path)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-09-01 12:56:40,884 - INFO - Starting MLflow run...
2025-09-01 12:56:41,461 - INFO - Logging preprocessing parameters...
2025-09-01 12:56:42,629 - INFO - Initializing Logistic Regression model...
2025-09-01 12:56:42,630 - INFO - Fitting the model...
2025-09-01 12:56:42,649 - INFO - Model training complete.
2025-09-01 12:56:42,650 - INFO - Logging model parameters...
2025-09-01 12:56:42,969 - INFO - Making predictions...
2025-09-01 12:56:42,971 - INFO - Calculating evaluation metrics...
2025-09-01 12:56:42,980 - INFO - Logging evaluation metrics...
2025-09-01 12:56:47,927 - INFO - Saving and logging the model...
2025/09/01 12:56:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-09-01 12:56:49,437 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\suref\AppData\Local\Temp\ipykernel_43300\2979654972.py", line 48, in 

🏃 View run trusting-boar-109 at: https://dagshub.com/sc/capstone.mlflow/#/experiments/0/runs/5bb648c7958a4d0db70155774efa3e0c
🧪 View experiment at: https://dagshub.com/sc/capstone.mlflow/#/experiments/0
